In [1]:
# Load dependencies for loading data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Import dependencies for pre-processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
data = pd.read_csv(r"C:\Users\capor\Downloads\archive\Predict Hair Fall.csv")

# Limpiar datos y categorizar

In [3]:
def one_hot_encode(data, columns):
    """
    Aplica One-Hot Encoding a las columnas especificadas de un DataFrame.
    
    Parámetros:
    data (pd.DataFrame): DataFrame de entrada.
    columns (list): Lista de nombres de columnas a codificar.
    
    Retorna:
    pd.DataFrame: DataFrame con las columnas codificadas y eliminadas las originales.
    """
    encoder = OneHotEncoder()
    
    for col in columns:
        encoded = encoder.fit_transform(data[[col]]).toarray()
        encoded_df = pd.DataFrame(encoded, columns=encoder.categories_[0])
        
        data = pd.concat([data, encoded_df], axis=1)
        data.drop(col, axis=1, inplace=True)
    
    return data

In [4]:
map_stres = {
    "Low": 0,
    "Moderate": 1,
    "High": 2
}
data["Stress"] = data["Stress"].map(map_stres)

In [5]:
def asignar_grupo(diccionario):
    """
    Devuelve una función que asigna un grupo basado en el valor encontrado en el diccionario.
    """
    def asignar(valor):
        for grupo, valores in diccionario.items():
            if valor in valores:
                return grupo
        return "Otros"  # Por seguridad
    return asignar

In [6]:
grupos_condiciones = {
        "Enfermedades inflamatorias de la piel": ["Eczema", "Psoriasis", "Dermatitis", "Seborrheic Dermatitis"],
        "Infecciones": ["Ringworm", "Scalp Infection"],
        "Trastornos del cabello": ["Alopecia Areata", "Androgenetic Alopecia"],
        "Problemas sistémicos": ["Thyroid Problems"],
        "Término genérico": ["Dermatosis"],
        "No Data": ["No Data"]
    }

In [7]:
grupos_deficiencias = {
        "Vitaminas": ["Vitamin A Deficiency", "Vitamin D Deficiency", "Biotin Deficiency", "Vitamin E deficiency"],
        "Minerales": ["Magnesium deficiency", "Selenium deficiency", "Zinc Deficiency"],
        "Macronutrientes": ["Protein deficiency"],
        "Ácidos grasos": ["Omega-3 fatty acids"],
        "Sin deficiencia": ["No Data", "Iron deficiency"]
    }

In [8]:
grupos_medicamentos = {
        "Antibióticos/Antifúngicos": ["Antibiotics", "Antifungal Cream"],
        "Enfermedades crónicas": ["Blood Pressure Medication", "Heart Medication"],
        "Inmunológicos": ["Immunomodulators", "Steroids"],
        "Salud mental": ["Antidepressants"],
        "Tratamientos para caída de cabello": ["Rogaine", "Accutane"],
        "Quimioterapia": ["Chemotherapy"],
        "No Data": ["No Data"]
    }

In [9]:
colums_yes_no = ["Genetics" ,"Hormonal Changes", "Poor Hair Care Habits ", "Environmental Factors", "Smoking", "Weight Loss "]

map_yes_no = {
    "Yes": 1,
    "No": 0
}

# Iterar por las columnas y mapear los valores
for col in colums_yes_no:
    if col in data.columns:  # Verifica que la columna exista
        data[col] = data[col].map(map_yes_no)
    else:
        print(f"Columna {col} no encontrada en el dataframe.")

In [10]:
exp11 = data.copy()
exp11["Medical Conditions"] = exp11["Medical Conditions"].str.strip()
exp11["Grupo_Condiciones"] = exp11["Medical Conditions"].apply(asignar_grupo(grupos_condiciones))
exp11.drop("Medical Conditions",axis=1,inplace=True)
exp11["Nutritional Deficiencies "] = exp11["Nutritional Deficiencies "].str.strip()
exp11["Grupo Deficiencias"] = exp11["Nutritional Deficiencies "].apply(asignar_grupo(grupos_deficiencias))
exp11.drop("Nutritional Deficiencies ",axis=1,inplace=True)
exp11["Medications & Treatments"] = exp11["Medications & Treatments"].str.strip()
exp11["Grupo Medicamentos"] = exp11["Medications & Treatments"].apply(asignar_grupo(grupos_medicamentos))
exp11.drop("Medications & Treatments",axis=1,inplace=True)

Comprobamos si ha cambiado

In [11]:
exp11

,Id,Genetics,Hormonal Changes,Stress,Age,Poor Hair Care Habits,Environmental Factors,Smoking,Weight Loss,Hair Loss,Grupo_Condiciones,Grupo Deficiencias,Grupo Medicamentos
0,133992,1,0,1,19,1,1,0,0,0,No Data,Minerales,No Data
1,148393,0,0,2,43,1,1,0,0,0,Enfermedades inflamatorias de la piel,Minerales,Antibióticos/Antifúngicos
2,155074,0,0,1,26,1,1,0,1,0,Término genérico,Macronutrientes,Antibióticos/Antifúngicos
3,118261,1,1,1,46,1,1,0,0,0,Infecciones,Vitaminas,Antibióticos/Antifúngicos
4,111915,0,0,1,30,0,1,1,0,1,Enfermedades inflamatorias de la piel,Sin deficiencia,Tratamientos para caída de cabello
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,184367,1,0,0,33,1,1,1,1,1,Enfermedades inflamatorias de la piel,Vitaminas,Tratamientos para caída de cabello
995,164777,1,1,0,47,0,0,0,1,0,No Data,Macronutrientes,Tratamientos para caída de cabello
996,143273,0,1,1,20,1,0,1,1,1,Trastornos del cabello,Macronutrientes,Salud mental
997,169123,0,1,1,32,1,1,1,1,1,Enfermedades inflamatorias de la piel,Vitaminas,Inmunológicos


In [12]:
# Crear los bins para los grupos de edad
bins = [0, 25, 35, 45, 60]

# Definir las etiquetas para cada grupo
labels = [0, 1, 2, 3]

# Crear la nueva columna con los grupos de edad
exp11['Age_group'] = pd.cut(exp11['Age'], bins=bins, labels=labels, right=False)


exp11.drop("Age", axis=1, inplace=True)

In [13]:
columns_onehotencoder = ["Grupo_Condiciones", "Grupo Deficiencias", "Grupo Medicamentos"]
exp11 = one_hot_encode(exp11, columns_onehotencoder)

In [14]:
from sklearn.model_selection import GridSearchCV
X = exp11.drop(["Hair Loss", "Id"], axis=1)
y = exp11["Hair Loss"]
    
sc = StandardScaler()
X = sc.fit_transform(X)

In [15]:
X

array([[ 0.95592526, -1.01920337,  0.0074575 , ..., -0.31465839,
        -0.3517592 , -0.52832691],
       [-1.04610689, -1.01920337,  1.24913072, ..., -0.31465839,
        -0.3517592 , -0.52832691],
       [-1.04610689, -1.01920337,  0.0074575 , ..., -0.31465839,
        -0.3517592 , -0.52832691],
       ...,
       [-1.04610689,  0.98115845,  0.0074575 , ..., -0.31465839,
         2.84285388, -0.52832691],
       [-1.04610689,  0.98115845,  0.0074575 , ..., -0.31465839,
        -0.3517592 , -0.52832691],
       [ 0.95592526,  0.98115845, -1.23421573, ..., -0.31465839,
        -0.3517592 , -0.52832691]])

# Entrenamiento modelo

In [24]:
gb = GradientBoostingClassifier(
    learning_rate=0.3,
    max_depth=3,
    min_samples_split=2,
    n_estimators=200,
    subsample=0.6
)

gb.fit(X, y)

# Evaluar el modelo con validación cruzada
cv_scores = cross_val_score(gb, X, y, scoring='f1_weighted', cv=5)

# Imprimir los resultados
print("Puntuaciones F1 (ponderadas) en validación cruzada:", cv_scores)
print("Media de la puntuación F1 (ponderada):", cv_scores.mean())

Puntuaciones F1 (ponderadas) en validación cruzada: [0.49939904 0.49481058 0.50439288 0.5547217  0.4897575 ]
Media de la puntuación F1 (ponderada): 0.5086163415753269


# Prueba datos ficticios

In [22]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

# Suponemos que ya tienes "gb" y "sc" entrenados

# Datos ficticios con las mismas columnas que en el preprocesamiento
datos_ficticios = {
    "Stress": ["Low"],
    "Age": [20],
    "Medical Conditions": ["Eczema"],
    "Nutritional Deficiencies ": ["Vitamin D Deficiency"],
    "Medications & Treatments": ["Rogaine"],
    "Genetics": ["No"],
    "Hormonal Changes": ["No"],
    "Poor Hair Care Habits ": ["No"],
    "Environmental Factors": ["Yes"],
    "Smoking": ["No"],
    "Weight Loss ": ["No"]
}

# Crear DataFrame
X_nuevo = pd.DataFrame(datos_ficticios)

# Mapear valores categóricos
map_stres = {"Low": 0, "Moderate": 1, "High": 2}
X_nuevo["Stress"] = X_nuevo["Stress"].map(map_stres)

map_yes_no = {"Yes": 1, "No": 0}
colums_yes_no = ["Genetics", "Hormonal Changes", "Poor Hair Care Habits ", "Environmental Factors", "Smoking", "Weight Loss "]
for col in colums_yes_no:
    X_nuevo[col] = X_nuevo[col].map(map_yes_no)

# Definir funciones de asignación
def asignar_grupo(valor, diccionario):
    for grupo, valores in diccionario.items():
        if valor in valores:
            return grupo
    return "Otros"

grupos_condiciones = {
    "Enfermedades inflamatorias de la piel": ["Eczema", "Psoriasis", "Dermatitis", "Seborrheic Dermatitis"],
    "Infecciones": ["Ringworm", "Scalp Infection"],
    "Trastornos del cabello": ["Alopecia Areata", "Androgenetic Alopecia"],
    "Problemas sistémicos": ["Thyroid Problems"],
    "Término genérico": ["Dermatosis"],
    "No Data": ["No Data"]
}
X_nuevo["Grupo_Condiciones"] = X_nuevo["Medical Conditions"].apply(lambda x: asignar_grupo(x, grupos_condiciones))
X_nuevo.drop("Medical Conditions", axis=1, inplace=True)

grupos_deficiencias = {
    "Vitaminas": ["Vitamin A Deficiency", "Vitamin D Deficiency", "Biotin Deficiency", "Vitamin E deficiency"],
    "Minerales": ["Magnesium deficiency", "Selenium deficiency", "Zinc Deficiency"],
    "Macronutrientes": ["Protein deficiency"],
    "Ácidos grasos": ["Omega-3 fatty acids"],
    "Sin deficiencia": ["No Data", "Iron deficiency"]
}
X_nuevo["Grupo Deficiencias"] = X_nuevo["Nutritional Deficiencies "].apply(lambda x: asignar_grupo(x, grupos_deficiencias))
X_nuevo.drop("Nutritional Deficiencies ", axis=1, inplace=True)

grupos_medicamentos = {
    "Antibióticos/Antifúngicos": ["Antibiotics", "Antifungal Cream"],
    "Enfermedades crónicas": ["Blood Pressure Medication", "Heart Medication"],
    "Inmunológicos": ["Immunomodulators", "Steroids"],
    "Salud mental": ["Antidepressants"],
    "Tratamientos para caída de cabello": ["Rogaine", "Accutane"],
    "Quimioterapia": ["Chemotherapy"],
    "No Data": ["No Data"]
}
X_nuevo["Grupo Medicamentos"] = X_nuevo["Medications & Treatments"].apply(lambda x: asignar_grupo(x, grupos_medicamentos))
X_nuevo.drop("Medications & Treatments", axis=1, inplace=True)

# Agrupar edad
bins = [0, 25, 35, 45, 60]
labels = [0, 1, 2, 3]
X_nuevo['Age_group'] = pd.cut(X_nuevo['Age'], bins=bins, labels=labels, right=False)
X_nuevo.drop("Age", axis=1, inplace=True)

# Aplicar One-Hot Encoding usando las mismas categorías del entrenamiento
columnas_categoricas = ["Grupo_Condiciones", "Grupo Deficiencias", "Grupo Medicamentos"]
X_nuevo = pd.get_dummies(X_nuevo, columns=columnas_categoricas)

# Asegurar que las columnas coincidan con el conjunto de entrenamiento
columnas_modelo = ['Genetics', 'Hormonal Changes', 'Stress',
       'Poor Hair Care Habits ', 'Environmental Factors', 'Smoking',
       'Weight Loss ', 'Age_group',
       'Enfermedades inflamatorias de la piel', 'Infecciones', 'No Data',
       'Problemas sistémicos', 'Trastornos del cabello', 'Término genérico',
       'Macronutrientes', 'Minerales', 'Sin deficiencia', 'Vitaminas',
       'Ácidos grasos', 'Antibióticos/Antifúngicos', 'Enfermedades crónicas',
       'Inmunológicos', 'No Data', 'Quimioterapia', 'Salud mental',
       'Tratamientos para caída de cabello']  # Asegúrate de definir las columnas originales de entrenamiento
X_nuevo = X_nuevo.reindex(columns=columnas_modelo, fill_value=0)

# Escalar los datos
X_nuevo = sc.transform(X_nuevo)

# Hacer la predicción
prediccion = gb.predict(X_nuevo)
print("La predicción para los datos ficticios es:", prediccion)

La predicción para los datos ficticios es: [0]


In [23]:
prediccion

array([0], dtype=int64)

In [24]:
X_nuevo

array([[-1.04610689, -1.01920337, -1.23421573, -0.98509604,  0.98312534,
        -1.03983172, -0.9463803 , -1.45557682, -0.73275035, -0.41702883,
        -0.3517592 , -0.33166248, -0.50812046, -0.31080081, -0.31465839,
        -0.61476096, -0.43344155, -0.79185594, -0.31848594, -0.48146919,
        -0.49091118, -0.4956167 , -0.04478859, -0.31465839, -0.3517592 ,
        -0.52832691]])

# Almacenar modelo

In [25]:
import pickle

# Guardar el modelo y el escalador en un archivo pickle
with open("modelo_calvicie.pkl", "wb") as f:
    pickle.dump({"modelo": gb, "escalador": sc}, f)

print("Modelo y escalador guardados en 'modelo_calvicie.pkl'")

Modelo y escalador guardados en 'modelo_calvicie.pkl'


In [26]:
with open("modelo_calvicie.pkl", "rb") as f:
    data = pickle.load(f)

modelo_cargado = data["modelo"]
escalador_cargado = data["escalador"]

print("Modelo y escalador cargados correctamente.")

Modelo y escalador cargados correctamente.


# Comprobación modelo

In [27]:
modelo_cargado

GradientBoostingClassifier(learning_rate=0.2, max_depth=7, subsample=0.8)

In [28]:
data

{'modelo': GradientBoostingClassifier(learning_rate=0.2, max_depth=7, subsample=0.8),
 'escalador': StandardScaler()}

In [41]:
a = sc.inverse_transform(X_nuevo)

In [44]:
b = sc.transform(X_nuevo)

c:\Users\capor\.conda\envs\Datathon\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [42]:
a

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.38777878e-17,  0.00000000e+00,
        -2.77555756e-17,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.38777878e-17,  0.00000000e+00]])

In [43]:
X_nuevo

array([[-1.04610689, -1.01920337, -1.23421573, -0.98509604,  0.98312534,
        -1.03983172, -0.9463803 , -1.45557682, -0.73275035, -0.41702883,
        -0.3517592 , -0.33166248, -0.50812046, -0.31080081, -0.31465839,
        -0.61476096, -0.43344155, -0.79185594, -0.31848594, -0.48146919,
        -0.49091118, -0.4956167 , -0.04478859, -0.31465839, -0.3517592 ,
        -0.52832691]])

# Comprobación del escalador

In [29]:
print(escalador_cargado.mean_)
print(escalador_cargado.var_)


[0.52252252 0.50950951 0.99399399 0.49249249 0.50850851 0.51951952
 0.47247247 1.47147147 0.34934935 0.14814815 0.11011011 0.0990991
 0.20520521 0.08808809 0.09009009 0.27427427 0.15815816 0.38538539
 0.09209209 0.18818819 0.19419419 0.1971972  0.002002   0.09009009
 0.11011011 0.21821822]
[0.24949274 0.24990957 0.64861258 0.24994364 0.24992761 0.24961899
 0.24924224 1.0219589  0.22730438 0.12620027 0.09798587 0.08927847
 0.16309603 0.08032858 0.08197387 0.1990479  0.13314416 0.23686349
 0.08361114 0.15277339 0.15648281 0.15831046 0.00199799 0.08197387
 0.09798587 0.17059903]
